In [1]:
import pandas as pd
import numpy as np
import re
import string
import random

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

import torch
from torch import nn
from torch.nn import functional as F

from sklearn.model_selection import train_test_split
from sklearn import metrics
import dlnlputils
from dlnlputils.data import tokenize_text_simple_regex, tokenize_corpus

# 1. Обработка данных

In [2]:
positive_dialogs_df = pd.read_excel(r'sentiment_train.xlsx', sheet_name='Positive', header=None, names=['data'])
negative_dialogs_df = pd.read_excel(r'sentiment_train.xlsx', sheet_name='Negative', header=None, names=['data'])

dialogs = pd.merge(positive_dialogs_df, negative_dialogs_df, how='outer')

targets_number = len(dialogs)//2
dialogs['target'] = np.append(np.ones(targets_number),np.zeros(targets_number))

dialogs = dialogs.sample(frac=1)

X_train, X_test, y_train, y_test = train_test_split(dialogs['data'], dialogs['target'])

# 2. Обучение модели

In [3]:
MAX_DF = 0.8   #токены, встречающиеся более чем в 80% документов выкидываются
MIN_COUNT = 3 #токеты встречающиеся меньше 3 раз выкидываются

sklearn_pipeline = Pipeline((('vect', TfidfVectorizer(tokenizer=tokenize_text_simple_regex,
                                                      max_df=MAX_DF,
                                                      min_df=MIN_COUNT)),
                             ('cls', LogisticRegression())))
sklearn_pipeline.fit(X_train, y_train);

# 3. Оценка качества

In [4]:
sklearn_train_pred = sklearn_pipeline.predict_proba(X_train)
sklearn_train_loss = F.cross_entropy(torch.from_numpy(sklearn_train_pred),
                                                 torch.from_numpy(np.array(y_train)).long())
print('Среднее значение функции потерь на обучении', float(sklearn_train_loss))
print('Доля верных ответов', accuracy_score(y_train, sklearn_train_pred.argmax(-1)))
print()

sklearn_test_pred = sklearn_pipeline.predict_proba(X_test)
sklearn_test_loss = F.cross_entropy(torch.from_numpy(sklearn_test_pred),
                                                torch.from_numpy(np.array(y_test)).long())
print('Среднее значение функции потерь на валидации', float(sklearn_test_loss))
print('Доля верных ответов', accuracy_score(y_test, sklearn_test_pred.argmax(-1)))

Среднее значение функции потерь на обучении 0.5934331769527817
Доля верных ответов 1.0

Среднее значение функции потерь на валидации 0.629271798019686
Доля верных ответов 1.0
